# CPID_EDC_Metrics — Aggregation & Qualification (Phase 1)

## Goal
Load CPID EDC metrics across all studies, normalize schemas,
clean columns, aggregate to subject/site level, and determine
whether this category is eligible for Phase 2 master-table integration.


In [1]:
# imports
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

# paths
DATA_DIR = Path("../data")
INTERMEDIATE_DIR = DATA_DIR / "intermediate"

inventory = pd.read_csv(INTERMEDIATE_DIR / "file_inventory.csv")


In [2]:
# Load CPID files
cpid_files = inventory[inventory.file_type == "cpid"]["file_path"]

assert len(cpid_files) > 0, "No CPID files found in inventory"

cpid_dfs = []

for f in cpid_files:
    f = Path(f)
    assert f.exists(), f"File not found: {f}"

    df = pd.read_excel(f)
    df["source_file"] = f.name
    cpid_dfs.append(df)

cpid_master = pd.concat(cpid_dfs, ignore_index=True)

assert cpid_master.shape[0] > 0, "CPID master dataframe is empty"

cpid_master.head()



,Project Name,Region,Country,Site ID,Subject ID,Latest Visit (SV) (Source: Rave EDC: BO4),Subject Status (Source: PRIMARY Form),Input files,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,CPMD,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,SSM,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,source_file
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Missing Visits,Missing Page,# Coded terms,# Uncoded Terms,# Open issues in LNR,# Open Issues reported for 3rd party reconcili...,Inactivated forms and folders,# eSAE dashboard review for DM,# eSAE dashboard review for safety,Visit status,Page status (Source: (Rave EDC : BO4)),NaN,NaN,NaN,NaN,Queries status (Source:(Rave EDC : BO4)),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Page Action Status (Source: (Rave EDC : BO4)),NaN,NaN,NaN,NaN,NaN,Protocol Deviations (Source:(Rave EDC : BO4)),NaN,PI Signatures (Source: (Rave EDC : BO4)),NaN,NaN,NaN,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,# Expected Visits (Rave EDC : BO4),# Pages Entered,# Pages with Non-Conformant data,# Total CRFs with queries & Non-Conformant data,# Total CRFs without queries & Non-Conformant ...,% Clean Entered CRF,# DM Queries,# Clinical Queries,# Medical Queries,# Site Queries,# Field Monitor Queries,# Coding Queries,# Safety Queries,#Total Queries,# CRFs Require Verification (SDV),# Forms Verified,# CRFs Frozen,# CRFs Not Frozen,# CRFs Locked,# CRFs Unlocked,# PDs Confirmed,# PDs Proposed,# CRFs Signed,CRFs overdue for signs within 45 days of Data ...,CRFs overdue for signs between 45 to 90 days o...,CRFs overdue for signs beyond 90 days of Data ...,Broken Signatures,CRFs Never Signed,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
2,Responsible LF for action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Site/CRA,NaN,NaN,NaN,DM,CSE/CDD,CDMD/Medical Lead,Site/CRA,CRA,Coder,Safety Team,NaN,CRA,NaN,NaN,DM,NaN,NaN,CD LF,NaN,Investigator,NaN,NaN,NaN,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
3,Study 5,ASIA,AUS,Site 2,Subject 2,Efficacy FU 1 (1),Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,92,0,NaN,NaN,NaN,0,3,0,0,0,0,0,3,12,71,0,0,0,0,5,0,81,1,0,0,1,0,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
4,Study 5,ASIA,AUS,Site 2,Subject 3,Efficacy FU 2 (1),Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,116,0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,6,24,0,0,0,0,0,0,107,0,0,0,0,0,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx


In [3]:
cpid_master.shape

(5515, 45)

In [4]:
cpid_master.columns.tolist()

['Project Name',
 'Region',
 'Country',
 'Site ID',
 'Subject ID',
 'Latest Visit (SV) (Source: Rave EDC: BO4)',
 'Subject Status (Source: PRIMARY Form)',
 'Input files',
 'Unnamed: 8',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'CPMD',
 'Unnamed: 17',
 'Unnamed: 18',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Unnamed: 22',
 'Unnamed: 23',
 'Unnamed: 24',
 'Unnamed: 25',
 'Unnamed: 26',
 'Unnamed: 27',
 'Unnamed: 28',
 'Unnamed: 29',
 'Unnamed: 30',
 'Unnamed: 31',
 'Unnamed: 32',
 'Unnamed: 33',
 'Unnamed: 34',
 'Unnamed: 35',
 'Unnamed: 36',
 'Unnamed: 37',
 'SSM',
 'Unnamed: 39',
 'Unnamed: 40',
 'Unnamed: 41',
 'Unnamed: 42',
 'Unnamed: 43',
 'source_file']

In [5]:
cpid_master.describe()

,Project Name,Region,Country,Site ID,Subject ID,Latest Visit (SV) (Source: Rave EDC: BO4),Subject Status (Source: PRIMARY Form),Input files,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,CPMD,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,SSM,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,source_file
count,5461,5439,5443,5461,5461,5385,5271,18,18,18,18,18,18,18,18,18,5497,5497,5497,18,18,18,5515,5497,5497,5497,5497,5497,5497,5479,5515,5479,5479,5497,5461,5461,5497,5461,5515,5479,5479,5479,5479,5479,5515
unique,19,3,58,816,4611,220,8,1,1,1,1,1,1,1,1,1,70,324,17,1,1,1,31,24,11,41,27,3,6,50,92,252,195,18,46,3,13,8,292,89,67,54,61,107,13
top,Study 25,EMEA,USA,Site 3,Subject 2,Screening,Screen Failure,Missing Visits,Missing Page,# Coded terms,# Uncoded Terms,# Open issues in LNR,# Open Issues reported for 3rd party reconcili...,Inactivated forms and folders,# eSAE dashboard review for DM,# eSAE dashboard review for safety,1,10,0,# Total CRFs with queries & Non-Conformant data,# Total CRFs without queries & Non-Conformant ...,% Clean Entered CRF,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CPID_EDC_Metrics_URSV2.0_updated.xlsx
freq,1374,2155,1007,224,10,2181,2232,18,18,18,18,18,18,18,18,18,2249,623,5266,18,18,18,4763,5144,5374,4708,5271,5461,5444,4100,2829,1279,3992,4883,5269,5437,4407,5248,725,3676,4928,5150,4273,3780,1648


In [6]:
cpid_master.describe().value_counts().sort_values()

Project Name  Region  Country  Site ID  Subject ID  Latest Visit (SV) (Source: Rave EDC: BO4)  Subject Status (Source: PRIMARY Form)  Input files     Unnamed: 8    Unnamed: 9     Unnamed: 10      Unnamed: 11           Unnamed: 12                                                  Unnamed: 13                    Unnamed: 14                     Unnamed: 15                         CPMD  Unnamed: 17  Unnamed: 18  Unnamed: 19                                      Unnamed: 20                                         Unnamed: 21          Unnamed: 22  Unnamed: 23  Unnamed: 24  Unnamed: 25  Unnamed: 26  Unnamed: 27  Unnamed: 28  Unnamed: 29  Unnamed: 30  Unnamed: 31  Unnamed: 32  Unnamed: 33  Unnamed: 34  Unnamed: 35  Unnamed: 36  Unnamed: 37  SSM   Unnamed: 39  Unnamed: 40  Unnamed: 41  Unnamed: 42  Unnamed: 43  source_file                          
19            3       58       816      4611        220                                        8                                      1               1

In [7]:
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
        .astype(str)
        .str.strip()
        .str.lower()
        .str.replace("%", "pct", regex=False)
        .str.replace("#", "num_", regex=False)
        .str.replace(r"[^\w]+", "_", regex=True)
        .str.replace(r"_+", "_", regex=True)
        .str.strip("_")
    )
    return df

cpid_master = normalize_columns(cpid_master)
cpid_master.head()

,project_name,region,country,site_id,subject_id,latest_visit_sv_source_rave_edc_bo4,subject_status_source_primary_form,input_files,unnamed_8,unnamed_9,unnamed_10,unnamed_11,unnamed_12,unnamed_13,unnamed_14,unnamed_15,cpmd,unnamed_17,unnamed_18,unnamed_19,unnamed_20,unnamed_21,unnamed_22,unnamed_23,unnamed_24,unnamed_25,unnamed_26,unnamed_27,unnamed_28,unnamed_29,unnamed_30,unnamed_31,unnamed_32,unnamed_33,unnamed_34,unnamed_35,unnamed_36,unnamed_37,ssm,unnamed_39,unnamed_40,unnamed_41,unnamed_42,unnamed_43,source_file
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Missing Visits,Missing Page,# Coded terms,# Uncoded Terms,# Open issues in LNR,# Open Issues reported for 3rd party reconcili...,Inactivated forms and folders,# eSAE dashboard review for DM,# eSAE dashboard review for safety,Visit status,Page status (Source: (Rave EDC : BO4)),NaN,NaN,NaN,NaN,Queries status (Source:(Rave EDC : BO4)),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Page Action Status (Source: (Rave EDC : BO4)),NaN,NaN,NaN,NaN,NaN,Protocol Deviations (Source:(Rave EDC : BO4)),NaN,PI Signatures (Source: (Rave EDC : BO4)),NaN,NaN,NaN,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,# Expected Visits (Rave EDC : BO4),# Pages Entered,# Pages with Non-Conformant data,# Total CRFs with queries & Non-Conformant data,# Total CRFs without queries & Non-Conformant ...,% Clean Entered CRF,# DM Queries,# Clinical Queries,# Medical Queries,# Site Queries,# Field Monitor Queries,# Coding Queries,# Safety Queries,#Total Queries,# CRFs Require Verification (SDV),# Forms Verified,# CRFs Frozen,# CRFs Not Frozen,# CRFs Locked,# CRFs Unlocked,# PDs Confirmed,# PDs Proposed,# CRFs Signed,CRFs overdue for signs within 45 days of Data ...,CRFs overdue for signs between 45 to 90 days o...,CRFs overdue for signs beyond 90 days of Data ...,Broken Signatures,CRFs Never Signed,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
2,Responsible LF for action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Site/CRA,NaN,NaN,NaN,DM,CSE/CDD,CDMD/Medical Lead,Site/CRA,CRA,Coder,Safety Team,NaN,CRA,NaN,NaN,DM,NaN,NaN,CD LF,NaN,Investigator,NaN,NaN,NaN,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
3,Study 5,ASIA,AUS,Site 2,Subject 2,Efficacy FU 1 (1),Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,92,0,NaN,NaN,NaN,0,3,0,0,0,0,0,3,12,71,0,0,0,0,5,0,81,1,0,0,1,0,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
4,Study 5,ASIA,AUS,Site 2,Subject 3,Efficacy FU 2 (1),Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,116,0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,6,24,0,0,0,0,0,0,107,0,0,0,0,0,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx


In [8]:
# drop unnamed columns
cpid_master = cpid_master.loc[
    :, ~cpid_master.columns.str.startswith("unnamed")
]

# drop all-null columns
cpid_master = cpid_master.dropna(axis=1, how="all")

# ensure no duplicate columns
assert cpid_master.columns.duplicated().sum() == 0, "Duplicate columns detected"

cpid_master.shape


(5515, 11)

In [9]:
cpid_master.columns.tolist()

['project_name',
 'region',
 'country',
 'site_id',
 'subject_id',
 'latest_visit_sv_source_rave_edc_bo4',
 'subject_status_source_primary_form',
 'input_files',
 'cpmd',
 'ssm',
 'source_file']

In [10]:
cpid_master["study_id"] = (
    cpid_master["project_name"]
    .astype(str)
    .str.strip()
)


In [11]:
assert cpid_master["study_id"].notna().all(), "Null study_id detected"
assert cpid_master["study_id"].nunique() > 1, "Only one study detected — verify inventory"


In [12]:
# extract study token from filename
cpid_master["study_id_from_file"] = (
    cpid_master["source_file"]
    .str.extract(r"(Study[\s_]*\d+)", expand=False)
)

# sanity check
mismatch = (
    cpid_master
    .dropna(subset=["study_id_from_file"])
    .query("study_id_from_file not in study_id")
)

assert mismatch.empty, "Study ID mismatch between project_name and filename"


In [13]:
CANONICAL_KEYS = [
    "study_id",
    "site_id",
    "subject_id"
]

missing = set(CANONICAL_KEYS) - set(cpid_master.columns)
assert not missing, f"Missing canonical keys: {missing}"


In [14]:
DESCRIPTIVE_COLS = [
    "project_name",
    "region",
    "country"
]

DESCRIPTIVE_COLS = [c for c in DESCRIPTIVE_COLS if c in cpid_master.columns]


In [15]:
# descriptive columns must be single-valued per canonical key
for col in DESCRIPTIVE_COLS:
    inconsistent = (
        cpid_master
        .groupby(CANONICAL_KEYS)[col]
        .nunique()
        .reset_index(name="nunique")
        .query("nunique > 1")
    )
    assert inconsistent.empty, f"Inconsistent values found in {col}"


In [16]:
SUM_COLS = [
    "input_files",
    "cpmd",
    "ssm"
]

SUM_COLS = [c for c in SUM_COLS if c in cpid_master.columns]


In [17]:
STATUS_COLS = [
    "latest_visit_sv_source_rave_edc_bo4",
    "subject_status_source_primary_form"
]

STATUS_COLS = [c for c in STATUS_COLS if c in cpid_master.columns]


In [18]:
for col in SUM_COLS:
    print(col, cpid_master[col].dtype)


input_files object
cpmd object
ssm object


In [19]:
NUMERIC_CAST_FAILURES = {}

for col in SUM_COLS:
    before_non_null = cpid_master[col].notna().sum()

    cpid_master[col] = pd.to_numeric(
        cpid_master[col],
        errors="coerce"
    )

    after_non_null = cpid_master[col].notna().sum()

    if after_non_null < before_non_null:
        NUMERIC_CAST_FAILURES[col] = before_non_null - after_non_null

In [20]:
NUMERIC_CAST_FAILURES


{'input_files': np.int64(18), 'cpmd': np.int64(36), 'ssm': np.int64(57)}

In [21]:
# Pre Aggregration checks
for col in SUM_COLS:
    assert (cpid_master[col].dropna() >= 0).all(), f"Negative values found in {col}"


In [22]:
for col, failed in NUMERIC_CAST_FAILURES.items():
    failure_rate = failed / len(cpid_master)
    assert failure_rate < 0.05, (
        f"Too many non-numeric values in {col}: {failure_rate:.2%}"
    )


In [23]:
assert cpid_master[CANONICAL_KEYS].isna().sum().sum() == 0, \
    "Null values detected in canonical keys"


AssertionError: Null values detected in canonical keys

In [24]:
cpid_master.shape

(5515, 13)

In [25]:
cpid_master[CANONICAL_KEYS].isna().sum()

study_id       0
site_id       54
subject_id    54
dtype: int64

In [26]:
cpid_master.columns.tolist()

['project_name',
 'region',
 'country',
 'site_id',
 'subject_id',
 'latest_visit_sv_source_rave_edc_bo4',
 'subject_status_source_primary_form',
 'input_files',
 'cpmd',
 'ssm',
 'source_file',
 'study_id',
 'study_id_from_file']

In [27]:
cpid_master.head()

,project_name,region,country,site_id,subject_id,latest_visit_sv_source_rave_edc_bo4,subject_status_source_primary_form,input_files,cpmd,ssm,source_file,study_id,study_id_from_file
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx,nan,Study 5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx,nan,Study 5
2,Responsible LF for action,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx,Responsible LF for action,Study 5
3,Study 5,ASIA,AUS,Site 2,Subject 2,Efficacy FU 1 (1),Follow-Up,NaN,10.0,81.0,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx,Study 5,Study 5
4,Study 5,ASIA,AUS,Site 2,Subject 3,Efficacy FU 2 (1),Follow-Up,NaN,14.0,107.0,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx,Study 5,Study 5


In [28]:
cpid_master.loc[
    cpid_master["subject_id"].isna(),
    ["study_id", "site_id", "subject_id", "source_file"]
].head(10)


,study_id,site_id,subject_id,source_file
0,nan,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
1,nan,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
2,Responsible LF for action,NaN,NaN,Study 5_CPID_EDC_Metrics_URSV2.0_updated.xlsx
682,nan,NaN,NaN,Study 10_CPID_EDC_Metrics_URSV2.0_14-Nov-2025_...
683,nan,NaN,NaN,Study 10_CPID_EDC_Metrics_URSV2.0_14-Nov-2025_...
684,Responsible LF for action,NaN,NaN,Study 10_CPID_EDC_Metrics_URSV2.0_14-Nov-2025_...
767,nan,NaN,NaN,Study 18_CPID_EDC_Metrics_URSV2.0_updated.xlsx
768,nan,NaN,NaN,Study 18_CPID_EDC_Metrics_URSV2.0_updated.xlsx
769,Responsible LF for action,NaN,NaN,Study 18_CPID_EDC_Metrics_URSV2.0_updated.xlsx
984,nan,NaN,NaN,Study 13_CPID_EDC_Metrics_URSV2.0_14NOV2025_up...


In [29]:
rows_before = len(cpid_master)
null_subject_rows = cpid_master["subject_id"].isna().sum()

# Guardrail: ensure this is a small fraction
assert null_subject_rows / rows_before < 0.02, (
    f"Too many rows without subject_id: {null_subject_rows} / {rows_before}"
)

cpid_master = cpid_master[cpid_master["subject_id"].notna()]

rows_after = len(cpid_master)
rows_before, rows_after


(5515, 5461)

In [30]:
assert cpid_master[CANONICAL_KEYS].isna().sum().sum() == 0, \
    "Null values detected in canonical keys"


In [32]:
# build aggregation dictionary
agg_dict = {}

# descriptive
for col in DESCRIPTIVE_COLS:
    agg_dict[col] = "first"

# numeric
for col in SUM_COLS:
    agg_dict[col] = "sum"

# status
for col in STATUS_COLS:
    agg_dict[col] = lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]


In [33]:
# build aggregation dict (as previously defined)
cpid_agg = (
    cpid_master
    .groupby(CANONICAL_KEYS, dropna=False)
    .agg(agg_dict)
    .reset_index()
)

# post-aggregation checks
assert cpid_agg.shape[0] > 0
assert cpid_agg.duplicated(CANONICAL_KEYS).sum() == 0
cpid_agg.shape

(5461, 11)

In [34]:
assert (
    cpid_agg.groupby("study_id").size().sum()
    <= cpid_master.groupby("study_id").size().sum()
), "Row inflation detected"


In [37]:
ID_COLS = ["study_id", "site_id", "subject_id"]

for col in ID_COLS:
    cpid_agg[col] = (
        cpid_agg[col]
        .astype(str)
        .str.strip()
    )

for col in ID_COLS:
    assert cpid_agg[col].map(type).eq(str).all(), f"Non-string values in {col}"


In [38]:
for col in DESCRIPTIVE_COLS:
    if col in cpid_agg.columns:
        cpid_agg[col] = cpid_agg[col].astype(str).str.strip()


In [39]:
cpid_agg.dtypes


study_id                                object
site_id                                 object
subject_id                              object
project_name                            object
region                                  object
country                                 object
input_files                            float64
cpmd                                   float64
ssm                                    float64
latest_visit_sv_source_rave_edc_bo4     object
subject_status_source_primary_form      object
dtype: object

In [41]:
out_parquet = INTERMEDIATE_DIR / "cpid_edc_metrics_agg.parquet"
out_csv = INTERMEDIATE_DIR / "cpid_edc_metrics_agg.csv"

cpid_agg.to_parquet(out_parquet, index=False)
cpid_agg.to_csv(out_csv, index=False)

(out_parquet, out_csv)


(PosixPath('../data/intermediate/cpid_edc_metrics_agg.parquet'),
 PosixPath('../data/intermediate/cpid_edc_metrics_agg.csv'))